In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import methylcheck

## Evaluation of the workflows  
The performance of the python and federated implmentation of the workflow provided by the x group at Exeter university are compared to the original r-version of this workflow.  
Additionally the performance of the federated workflow is tested in several cases of sample size and class label imbalance.

In [47]:
GSE66351_pheno = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE66351\\Reduced_Pheno_Info.csv", index_col=0)
GSE105109_pheno = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE105109\\GSE105109_Reduced_Pheno_Info.csv", index_col=0)
GSE134379_pheno = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE134379\\Reduced_Pheno_Info.csv", index_col=0)
# make sure all the phenotype files used the same codes for the diagnosis
GSE105109_pheno.loc[GSE105109_pheno["Diagnosis"] == " Control", "Diagnosis"] = " CTRL"
GSE105109_pheno.loc[GSE105109_pheno["Diagnosis"] == " Alzheimer's disease", "Diagnosis"] = " AD"

GSE134379_pheno.loc[GSE134379_pheno["Diagnosis"] == " ND", "Diagnosis"] = " CTRL"
GSE134379_pheno.loc[GSE134379_pheno["Diagnosis"] == " AD", "Diagnosis"] = " AD"

# recode the sex column so in all datasets "gender: F" is females and "gender: M" is males
# only GSE66351 needs to be recoded that way

GSE66351_pheno.loc[GSE66351_pheno["Sex"] == " F", "Sex"] = " F"
GSE66351_pheno.loc[GSE66351_pheno["Sex"] == " M", "Sex"] = " M"

In [2]:
# read in the results from the different implementations
original_r_GSE66351_E = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE66351\\EWAS\\Results_small_dataset.csv", index_col=0)
#original_r_GSE66351_B = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE66351\\Normalised_Betas.csv", index_col=0)


original_r_GSE105109_E = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE105109\\EWAS\\Results_small_dataset.csv", index_col=0)
#original_r_GSE105109_B = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE105109\\GSE105109_Normalised_Betas.csv", index_col=0)


original_r_GSE134379_E = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE134379\\EWAS\\Results_small_dataset.csv", index_col=0)
#original_r_GSE134379_B = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE134379\\Normalised_Betas.csv", index_col=0)


original_r_combined = None

In [34]:
python_central_GSE66351_E = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\QC_Python\\GSE66351results_diagnosis_regression_python.csv", index_col=0)
python_central_GSE66351_B = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\QC_Python\\GSE66351_normalised_betas_python.csv", index_col=0)

python_central_GSE105109 = None
python_central_GSE134379 = None
python_combined = None

In [9]:
federated = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_Fed\\_even_Results_EWAS1.csv", index_col=0, header=[0,1])


In [10]:
federated_mild_imbalance = pd.read_csv("E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_Fed\_mild_Results_EWAS1.csv", index_col=0, header = [0,1])
federated_strong_imbalance = pd.read_csv("E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_Fed\_strong_Results_EWAS1.csv", index_col=0, header = [0,1])

In [5]:
def calculate_GroudTruth(R_EWAS_results, diff_meth_thresh = 0.05):
    #control_probe_means = Normalised_betas.loc[:, phenotype.index[phenotype["Diagnosis"] == " CTRL"]].mean(axis=1)
    #AD_probe_means = Normalised_betas.loc[:, phenotype.index[phenotype["Diagnosis"] != " CTRL"]].mean(axis=1)
    probe_difference = pd.Series.abs(R_EWAS_results.loc[:,"Diagnosis_Beta"])
    DM_probes = probe_difference[probe_difference >= diff_meth_thresh]
    not_DM_probes = probe_difference[probe_difference < diff_meth_thresh]

    # selecting the significant probes from the EWAS results
    EWAS_sig = R_EWAS_results.loc[R_EWAS_results["corr_pval"] <= 0.05, :]
    EWAS_notsig = R_EWAS_results.loc[R_EWAS_results["corr_pval"] > 0.05, :]

    # true positives - EWAS significant and differentially methylated
    true_positive = set(DM_probes.index.values).intersection(set(EWAS_sig.index.values))
    # true negatives - EWAS not significant and not differentially methylated
    true_negative = set(not_DM_probes.index.values).intersection(set(EWAS_notsig.index.values))
    # false positives - EWAS significant and not differentially methylated
    false_positve = set(not_DM_probes.index.values).intersection(set(EWAS_sig.index.values))
    # false negatives = EWAS not significant and differentially methylated
    false_negative = set(DM_probes.index.values).intersection(set(EWAS_notsig.index.values))

    return {"TP":true_positive, "TN":true_negative, "FP":false_positve, "FN":false_negative}

    

In [ ]:
def calculate_EWAS_metrics_p(EWAS_results, ground_truth, diff_meth_thresh = 0.05,  multiIndex = False):
    #control_probe_means = Normalised_betas.loc[:, phenotype.index[phenotype["Diagnosis"] == " CTRL"]].mean(axis=1)
    #AD_probe_means = Normalised_betas.loc[:, phenotype.index[phenotype["Diagnosis"] != " CTRL"]].mean(axis=1)
    if multiIndex:
            probe_difference = pd.Series.abs(EWAS_results.loc[:, ("Methylation change", "Diagnosis")])
    else:
        probe_difference = pd.Series.abs(EWAS_results.loc[:,"Methylation Change"])
    DM_probes = probe_difference[probe_difference >= diff_meth_thresh]
    not_DM_probes = probe_difference[probe_difference < diff_meth_thresh]

    # selecting the significant probes from the EWAS results
    if multiIndex:
            EWAS_sig = EWAS_results.loc[EWAS_results[("Corrected P-value", "Diagnosis")] <= 0.05, :]
    else:
        EWAS_sig = EWAS_results.loc[EWAS_results["Corrected P-value"] <= 0.05, :]
    
    if multiIndex:
            EWAS_notsig = EWAS_results.loc[EWAS_results[("Corrected P-value", "Diagnosis")] > 0.05, :]
    else:
        EWAS_notsig = EWAS_results.loc[EWAS_results["Corrected P-value"] > 0.05, :]

    # true positives - EWAS significant and differentially methylated in python/fed and in original R
    true_positive = len(set(set(DM_probes.index.values).intersection(set(EWAS_sig.index.values))).intersection(ground_truth["TP"]))
    # true negatives - EWAS not significant and not differentially methylated in python/fed and in original R
    true_negative = len(set(set(not_DM_probes.index.values).intersection(set(EWAS_notsig.index.values))).intersection(ground_truth["TN"]))
    # false positives - EWAS significant and not differentially methylated in python/fed and in original R
    false_positve = len(set(set(not_DM_probes.index.values).intersection(set(EWAS_sig.index.values))).intersection(ground_truth["FP"]))
    # false negatives = EWAS not significant and differentially methylated in python/fed and in original R
    false_negative = len(set(set(DM_probes.index.values).intersection(set(EWAS_notsig.index.values))).intersection(ground_truth["FN"]))

    Acc = (true_positive + true_negative)/(true_positive+true_negative+false_positve+false_negative)
    if (true_positive+false_positve) > 0:
        Pre = true_positive/(true_positive+false_positve)
    else:
        Pre = 0
    if (true_positive+false_negative) > 0:
        Rec = true_positive/(true_positive + false_negative)
    else:
        Rec = 0
    if Pre and Rec:
        F1 = 2*(Rec*Pre)/(Rec + Pre)
    else:
        F1 = 0

    return {"TP":true_positive, "TN":true_negative, "FP":false_positve, "FN":false_negative, "Accuracy":Acc, "Precision":Pre, "Recall":Rec, "F1":F1}


In [13]:
def groundTruthNew(R_EWAS_results, diff_meth_thresh = 0.05):
    probe_difference = pd.Series.abs(R_EWAS_results.loc[:,"Diagnosis_Beta"])
    DM_probes = probe_difference[probe_difference >= diff_meth_thresh].index.values
    DM_probes = set(DM_probes).intersection(set(R_EWAS_results.loc[R_EWAS_results["corr_pval"] <= 0.05, :].index.values))
    notDM_probes = set(R_EWAS_results.index.values).difference(DM_probes)
    return {"DM":DM_probes, "Not DM":notDM_probes}

In [54]:
def EWAS_metrics_p(EWAS_results, ground_truth, diff_meth_thresh = 0.05,  multiIndex = False):
    #control_probe_means = Normalised_betas.loc[:, phenotype.index[phenotype["Diagnosis"] == " CTRL"]].mean(axis=1)
    #AD_probe_means = Normalised_betas.loc[:, phenotype.index[phenotype["Diagnosis"] != " CTRL"]].mean(axis=1)
    if multiIndex:
            probe_difference = pd.Series.abs(EWAS_results.loc[:, ("Methylation change", "Diagnosis")])
            EWAS_sig = EWAS_results.loc[EWAS_results[("Corrected P-value", "Diagnosis")] <= 0.05, :].index
            
    else:
        probe_difference = pd.Series.abs(EWAS_results.loc[:,"Methylation Change"])
        EWAS_sig = EWAS_results.loc[EWAS_results["Corrected P-value"] <= 0.05, :].index
        
    
    good_probes = probe_difference[probe_difference >= diff_meth_thresh].index
    good_probes = set(good_probes).intersection(set(EWAS_sig))
    not_good = set(EWAS_results.index.values).difference(good_probes)

    
    # true positives - EWAS significant and differentially methylated in python/fed and in original R
    true_positive = len(ground_truth["DM"].intersection(good_probes))
    # true negatives - EWAS not significant and not differentially methylated in python/fed and in original R
    true_negative = len(ground_truth["Not DM"].intersection(not_good))
    # false positives - EWAS significant and not differentially methylated in python/fed and in original R
    false_positve = len(ground_truth["Not DM"].intersection(good_probes))
    # false negatives = EWAS not significant and differentially methylated in python/fed and in original R
    false_negative = len(ground_truth["DM"].intersection(not_good))

    Acc = (true_positive + true_negative)/(true_positive+true_negative+false_positve+false_negative)
    if (true_positive+false_positve) > 0:
        Pre = true_positive/(true_positive+false_positve)
    else:
        Pre = 0
    if (true_positive+false_negative) > 0:
        Rec = true_positive/(true_positive + false_negative)
    else:
        Rec = 0
    if Pre and Rec:
        F1 = 2*(Rec*Pre)/(Rec + Pre)
    else:
        F1 = 0

    return {"TP":true_positive, "TN":true_negative, "FP":false_positve, "FN":false_negative, "Accuracy":Acc, "Precision":Pre, "Recall":Rec, "F1":F1}


In [15]:
GSE66351_groundTruth = groundTruthNew(original_r_GSE66351_E)

In [56]:
# loop through different methylation cut-offs to compare perfomance of R and fed
thresholds = {"0.01":0.01, "0.05":0.05, "0.1":0.1, "0.15":0.15, "0.2":0.20}
datasets = {"No Inbalance":federated, "Mild Inbalance":federated_mild_imbalance, "Strong Inbalance":federated_strong_imbalance}
results = {}
#thesis_tables = pd.ExcelWriter("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\ThesisTable.xlsx", mode="a", engine="openpyxl")
for data in datasets:
    inner = {}
    for i in thresholds:
        metrics = EWAS_metrics_p(datasets[data], diff_meth_thresh=thresholds[i], ground_truth=GSE66351_groundTruth, multiIndex=True)
        inner[i] = metrics
    results[data] = pd.DataFrame.from_dict(inner).T
    # save the metric tables to excel
    with pd.ExcelWriter("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\ThesisTable.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        results[data].to_excel(writer, sheet_name = (data + "EWASMetrics"))

    

In [55]:
GSE66351_metrics_p = EWAS_metrics_p(python_central_GSE66351_E, diff_meth_thresh=0.05, ground_truth = GSE66351_groundTruth, multiIndex=False)
federated_metrics = EWAS_metrics_p(federated, diff_meth_thresh=0.05, ground_truth = GSE66351_groundTruth, multiIndex=True)
# combine the metrics results into one dataframe
performance_results = pd.DataFrame.from_dict([GSE66351_metrics_p, federated_metrics])
performance_results.index = ["GSE66351 Pyhton central", "GSE66351 Federated"]
#performance_results.to_excel("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\ThesisTables.xlsx")

performance_results

,TP,TN,FP,FN,Accuracy,Precision,Recall,F1
GSE66351 Pyhton central,2,450789,38,8,0.999898,0.05,0.2,0.08
GSE66351 Federated,0,450827,0,10,0.999978,0.00,0.0,0.00


In [41]:
GSE66351_groundTruth["DM"].intersection(set(python_central_GSE66351_E.loc[python_central_GSE66351_E.loc[:,"Corrected P-value"] <= 0.05,:])) 

set()

In [51]:
len(set(python_central_GSE66351_E.loc[python_central_GSE66351_E.loc[:,"Corrected P-value"] <= 0.05].index))

40

In [43]:
len(GSE66351_groundTruth["DM"])

10